In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import numpy as np
import re


In [ ]:
import pandas as pd

url = "/content/enronSpamSubset.csv"

df = pd.read_csv(url)



In [ ]:
dataset = df[['Body', 'Label']]
dataset. dropna() 
dataset[dataset["Label"] == 0].shape

(5000, 2)

In [ ]:
dataset['Body'] = dataset['Body'].apply (lambda x: " ".join (x.lower () for x in x.split ()))

In [ ]:
dataset['Body'] = dataset['Body'].str.replace (r"""[^\w\s]+""","", regex = True)

dataset['Body'][1]

sample_dataset = dataset

In [ ]:
import nltk
from nltk.corpus import stopwords


nltk.download("stopwords")
# remove stop words
stop = stopwords.words ('english')
sample_dataset['Body']= sample_dataset['Body'].apply (lambda x: " ".join (x for x in x.split () if x not in stopwords.words("english")))
sample_dataset['Body']

In [ ]:
sample_dataset.to_csv('clean_dataset.csv')

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,x_test = train_test_split(sample_dataset['Body'],sample_dataset['Label'],stratify=sample_dataset['Label'])
#print(x_train.shape,y_train.shape)


In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4", trainable=True)

In [ ]:
dataset_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="input")
preprocessed_text = bert_preprocess(dataset_input)
word_embedded = bert_encoder(preprocessed_text)

layer = tf.keras.layers.Dropout(0.1, name="dropout")(word_embedded['pooled_output'])
layer = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(layer)

model = tf.keras.Model(inputs=[dataset_input], outputs = [layer])


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['input[0][0]']                  
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [ ]:
model.fit(x_train,y_train, epochs=50)

Epoch 1/50
3/3 [==============================] - 101s 28s/step - loss: 0.7928 - accuracy: 0.4933 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 2/50
3/3 [==============================] - 98s 28s/step - loss: 0.7215 - accuracy: 0.4667 - precision: 0.4706 - recall: 0.6486
Epoch 3/50
3/3 [==============================] - 95s 27s/step - loss: 0.7562 - accuracy: 0.5200 - precision: 0.5072 - recall: 0.9459
Epoch 4/50
3/3 [==============================] - 96s 28s/step - loss: 0.7228 - accuracy: 0.4933 - precision: 0.4800 - recall: 0.3243
Epoch 5/50
3/3 [==============================] - 101s 28s/step - loss: 0.7326 - accuracy: 0.4667 - precision: 0.4545 - recall: 0.4054
Epoch 6/50
3/3 [==============================] - 96s 27s/step - loss: 0.7444 - accuracy: 0.5333 - precision: 0.5139 - recall: 1.0000
Epoch 7/50
3/3 [==============================] - 97s 28s/step - loss: 0.7189 - accuracy: 0.5733 - precision: 0.5758 - recall: 0.5135
Epoch 8/50
3/3 [==============================] - 96

In [ ]:
!mkdir -p saved_model
model.save('saved_model/my_model')


In [ ]:
new_model = tf.keras.models.load_model('saved_model/my_model')

In [ ]:
dataset_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="input")
preprocessed_text = bert_preprocess(x_train)
word_embedded = bert_encoder(preprocessed_text)
